In [1]:
import timeit
import asyncio

from aiohttp import ClientSession

from web3.providers.base import JSONBaseProvider
from web3.providers import HTTPProvider
from web3 import Web3

In [2]:
# ipykernels have asyncio always running in the back. need to do this for other asyncios in the backend
import nest_asyncio
nest_asyncio.apply()

In [3]:
# synchronously request receipts for given transactions
def sync_receipts(web3, transactions):
    for tran in transactions:
        web3.eth.getTransactionReceipt(tran)

In [4]:
# asynchronous JSON RPC API request
async def async_make_request(session, url, method, params):
    base_provider = JSONBaseProvider()
    request_data = base_provider.encode_rpc_request(method, params)
    async with session.post(url, data=request_data,
                        headers={'Content-Type': 'application/json'}) as response:
        content = await response.read()
    response = base_provider.decode_rpc_response(content)
    return response

In [5]:
async def run(node_address, transactions):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for tran in transactions:
            task = asyncio.ensure_future(async_make_request(session, node_address,
                                                            'eth_getTransactionReceipt',[tran.hex()]))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)

In [6]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
eth_node_address = f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"
web3 = Web3(HTTPProvider(eth_node_address))

In [7]:
block = web3.eth.getBlock(web3.eth.blockNumber)
transactions = block['transactions']

In [8]:
start_time = timeit.default_timer()
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(eth_node_address, transactions))
result = loop.run_until_complete(future)
print('async: {:.3f}s'.format(timeit.default_timer() - start_time))

async: 5.068s


In [9]:
start_time = timeit.default_timer()
sync_receipts(web3, transactions)
print('sync: {:.3f}s'.format(timeit.default_timer() - start_time))

sync: 81.911s
